In [1]:
%load_ext autoreload
# reloads the file constantly so you don't need to reset
%autoreload 2

In [1]:
from nn.nn import NeuralNetwork
from nn.preprocess import one_hot_encode_seqs, sample_seqs
#import io
from nn.io import read_fasta_file, read_text_file
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

/Users/yaenchen/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [7]:
np.zeros(3)

array([0., 0., 0.])

In [19]:
pos = open('data/rap1-lieb-positives.txt').read().split("\n")
neg = read_fasta_file('data/yeast-upstream-1k-negative.fa')

In [ ]:
# output should match A_prev
# number of neurons in this layer is the number of biases in the bias matrix
num_neurons = len(b_curr)
# number of outputs is the number of neurons
A_curr = np.zeros(num_neurons)
for i in range(0, num_neurons):
    output = A_prev * W_curr + b_curr

In [134]:
# Create network
layers = [{"input_dim": 68, "output_dim": 34, "activation": "sigmoid"},
          {"input_dim": 34, "output_dim": 17, "activation": "sigmoid"},
          {"input_dim": 17, "output_dim": 1, "activation": "sigmoid"},]
network = NeuralNetwork(layers, lr = 0.01, seed = 42, batch_size = 100, epochs = 20, loss_function = "bce")

[print(index, layer) for index, layer in enumerate(network.arch)]

0 {'input_dim': 68, 'output_dim': 34, 'activation': 'sigmoid'}
1 {'input_dim': 34, 'output_dim': 17, 'activation': 'sigmoid'}
2 {'input_dim': 17, 'output_dim': 1, 'activation': 'sigmoid'}


[None, None, None]

In [52]:
digits = load_digits()
print("Shape of the digits data:", np.shape(digits.data))
X_train, X_val, y_train, y_val = train_test_split(digits["data"], digits["target"], test_size = 0.2, random_state = 0)

Shape of the digits data: (1797, 64)


In [153]:
def forward(network, X: ArrayLike):
    """
    This method is responsible for one forward pass of the entire neural network.

    Args:
        X: ArrayLike
            Input matrix with shape [batch_size, features].

    Returns:
        output: ArrayLike
            Output of forward pass.
        cache: Dict[str, ArrayLike]:
            Dictionary storing Z and A matrices from `_single_forward` for use in backprop.
    """
    # initialize cache with the input matrix (will be changed later, just need a matrix in there)
    cache = {'Z0':X}
    
    for (index, layer) in enumerate(network.arch):
        print(index, layer)
        # for each layer, get the weights, bias, and activation function
        # index starts from 0 but dictionary references beginning from 1, so add 1 to index
        W_curr = network._param_dict[f"W{index + 1}"]
        b_curr = network._param_dict[f"b{index + 1}"]
        activation = layer['activation']
        
        # forward for that layer
        A_curr, Z_curr = self._single_forward(W_curr, b_curr, A_prev, activation)
        # store new A and Z to cache dictionary
        cache[f"A{layer_idx}"] = A_curr
        cache[f"Z{layer_idx}"] = Z_curr
        
        # initialize current A as previous A for the next layer iteration
        A_prev = A_curr
    
    # After all the layers
    return A_prev, cache
    

In [154]:
forward(network, X_train)
#for i, l in (list(enumerate(net.arch))):
#    print(i, l)

0 {'input_dim': 68, 'output_dim': 34, 'activation': 'sigmoid'}
sigmoid
1 {'input_dim': 34, 'output_dim': 17, 'activation': 'sigmoid'}
sigmoid
2 {'input_dim': 17, 'output_dim': 1, 'activation': 'sigmoid'}
sigmoid


In [38]:
        # Initialize cache and set first A_prev
        cache = {"A0": X}
        A_prev = X

        # Go through each layer
        for idx, layer in enumerate(self.arch):

            # "Actual" idx is one higher to correspond to _init_params definition
            layer_idx = idx + 1

            # Get info for this layer
            W_curr = self._param_dict[f"W{layer_idx}"]
            b_curr = self._param_dict[f"b{layer_idx}"]
            activation = layer["activation"]

            # Do the forward pass
            A_curr, Z_curr = self._single_forward(W_curr, b_curr, A_prev, activation)

            # Store matrices in cache and set up next loop
            cache[f"A{layer_idx}"] = A_curr
            cache[f"Z{layer_idx}"] = Z_curr
            A_prev = A_curr

        # Return final activation and cache
        return A_prev, cache

t
e
s
t


[None, None, None, None]

In [157]:
1 / (1 + np.e ** (-100)) == 1 / (1 + np.exp(-100))

True

In [196]:
import numpy as np
from typing import List, Tuple
from numpy.typing import ArrayLike
from collections import Counter
from sklearn.utils import shuffle

def sample_seqs(seqs: List[str], labels: List[bool]) -> Tuple[List[str], List[bool]]:
    """
    This function should sample the given sequences to account for class imbalance. 
    Consider this a sampling scheme with replacement.
    
    Args:
        seqs: List[str]
            List of all sequences.
        labels: List[bool]
            List of positive/negative labels

    Returns:
        sampled_seqs: List[str]
            List of sampled sequences which reflect a balanced class size
        sampled_labels: List[bool]
            List of labels for the sampled sequences
    """
    # get the total number of entries
    num_entries = len(seqs)
    # the balance class size is the total number of entries divided by the number of classes for equal split
    # only 2 classes: positive and negative
    balanced_size = int(np.ceil(num_entries / 2))

    # get the count of class frequencies
    label_counts = Counter(labels).values()
    # get the less frequent class name
    less_frequent_label = list(Counter(labels).keys())[1]
    # get pos class indices
    pos_indices = [i for i, x in enumerate(labels) if x == "True"]
    # get neg class
    neg_indices = [i for i, x in enumerate(labels) if x == "False"]
    # resample using random indices
    # get random indices from total positive and negative indices lists
    print(pos_indices)
    print(neg_indices)
    pos_sample_indices = np.random.choice(pos_indices, size=balanced_size, replace=True)
    neg_sample_indices = np.random.choice(neg_indices, size=balanced_size, replace=True)
    # get sequences from the random indices
    sampled_seqs = [seqs[i] for i in pos_sample_indices] + [seqs[i] for i in neg_sample_indices]
    # get labels from the random indices
    sampled_labels = [labels[i] for i in pos_sample_indices] + [labels[i] for i in neg_sample_indices]
    # shuffle the final sequences and labels in the same manner to keep position consistent
    sampled_seqs, sampled_labels = shuffle(sampled_seqs, sampled_labels)
    
    return sampled_seqs, sampled_labels

In [60]:
sample_seqs(['A', 'A', 'A', 'B'], [True,True,True,False])

(['A', 'A', 'B', 'B'], [True, True, False, False])

In [220]:
from itertools import chain
def one_hot_encode_seqs(seq_arr: List[str]) -> ArrayLike:
    """
    This function generates a flattened one-hot encoding of a list of DNA sequences
    for use as input into a neural network.

    Args:
        seq_arr: List[str]
            List of sequences to encode.

    Returns:
        encodings: ArrayLike
            Array of encoded sequences, with each encoding 4x as long as the input sequence.
            For example, if we encode:
                A -> [1, 0, 0, 0]
                T -> [0, 1, 0, 0]
                C -> [0, 0, 1, 0]
                G -> [0, 0, 0, 1]
            Then, AGA -> [1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0].
    """
    encodings = []

    # iterate through nucleotides
    # append to final encodings list the new one-hot encoded values
    for nuc in seq_arr:
        if nuc == 'A':
            encodings.append([1, 0, 0, 0])
        elif nuc == 'T':
            encodings.append([0, 1, 0, 0])
        elif nuc == 'C':
            encodings.append([0, 0, 1, 0])
        elif nuc == 'G':
            encodings.append([0, 0, 0, 1])

    # return final encodings as an array
    return list(chain.from_iterable(np.array(encodings)))

In [221]:
one_hot_encode_seqs('AGA')

[1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0]

In [34]:
net = NeuralNetwork([{'input_dim': 5, 'output_dim': 10, 'activation': "sigmoid"},
                        {'input_dim': 10, 'output_dim': 5, 'activation': "sigmoid"}],
                        lr=0.1,
                        seed=10,
                        batch_size=5,
                        epochs=5,
                        loss_function='bce')

In [11]:
np.random.seed(10)
X = np.random.randn(10, 5) # 3 observations, 4 features

In [12]:
X 

array([[ 1.3315865 ,  0.71527897, -1.54540029, -0.00838385,  0.62133597],
       [-0.72008556,  0.26551159,  0.10854853,  0.00429143, -0.17460021],
       [ 0.43302619,  1.20303737, -0.96506567,  1.02827408,  0.22863013],
       [ 0.44513761, -1.13660221,  0.13513688,  1.484537  , -1.07980489],
       [-1.97772828, -1.7433723 ,  0.26607016,  2.38496733,  1.12369125],
       [ 1.67262221,  0.09914922,  1.39799638, -0.27124799,  0.61320418],
       [-0.26731719, -0.54930901,  0.1327083 , -0.47614201,  1.30847308],
       [ 0.19501328,  0.40020999, -0.33763234,  1.25647226, -0.7319695 ],
       [ 0.66023155, -0.35087189, -0.93943336, -0.48933722, -0.80459114],
       [-0.21269764, -0.33914025,  0.31216994,  0.56515267, -0.14742026]])

In [61]:
output, cache = net.forward(X)

In [63]:
cache

{'A0': array([[-0.34888192, -0.43902624,  0.14110417,  0.27304932, -1.61857075],
        [-0.57311336, -1.32044755,  1.23620533,  2.46532508,  1.38323223],
        [ 0.34623312,  1.02251611,  0.16681027,  1.65671662,  0.66788961],
        [-0.22994664, -1.12955119, -0.6399626 ,  0.31383052, -1.22583598],
        [-0.22179314,  1.33992631,  0.02930971,  1.98538575,  1.4471656 ],
        [-0.28762941, -1.35931057, -0.04804133, -0.48078734,  0.37775309],
        [ 1.61440797, -1.12310404, -0.38872795,  0.33234995,  1.13497317],
        [ 0.51071441,  0.41429764,  1.34454942,  0.49351532, -0.23700418],
        [ 0.05728515, -0.70707145,  0.54666484,  0.94250041, -2.97959677],
        [ 1.21814885, -0.05652072,  0.46088845,  0.66237401, -2.29510333]]),
 'A1': array([[0.44941055, 0.51806323, 0.46394821, 0.57990949, 0.52915659,
         0.46772466, 0.51249705, 0.55364032, 0.5211759 , 0.50744671],
        [0.43032647, 0.50636081, 0.4820492 , 0.6055673 , 0.7692477 ,
         0.52552173, 0.60044

In [20]:
net = NeuralNetwork([{'input_dim': 1, 'output_dim': 2, 'activation': "sigmoid"},
                        {'input_dim': 2, 'output_dim': 1, 'activation': "sigmoid"}],
                        lr=0.1,
                        seed=10,
                        batch_size=1,
                        epochs=2,
                        loss_function='bce')


In [35]:
A, Z = net._single_forward(W_curr = np.array([1, 2, 3]),
                           b_curr=np.array([1, 2, 3]),
                          A_prev=np.array([1, 0, 1]),
                          activation=net.arch[0]['activation'])

In [40]:
np.random.seed(10)
# instantiate simple neural network
net = NeuralNetwork([{'input_dim': 5, 'output_dim': 10, 'activation': "sigmoid"},
                        {'input_dim': 10, 'output_dim': 5, 'activation': "sigmoid"}],
                        lr=0.1,
                        seed=10,
                        batch_size=5,
                        epochs=5,
                        loss_function='bce')
# make some data
X = np.random.randn(10, 5)

 
A, Z = net._single_forward(W_curr=np.array([1, 2, 3]),
                           b_curr=np.array([1, 2, 3]),
                           A_prev=np.array([1, 0, 1]),
                           activation=net.arch[0]['activation'])
# assert statements on example
assert np.allclose(A, np.array([0.99330715, 0.99752738, 0.99908895]))
assert np.allclose(Z, np.array([5, 6, 7]))

In [47]:
W_curr = np.array([1, 2, 3])
b_curr = np.array([1, 2, 3])
A_prev = np.array([1, 0, 1])
Z_curr = np.array([5, 6, 7])
dA_curr = np.array([[1, 1, 1],
                    [1, 1, 1],
                    [1, 1, 1]])
dA_prev, dW_curr, db_curr = net._single_backprop(W_curr, b_curr, Z_curr, A_prev, dA_curr, net.arch[0]['activation'])

In [50]:
db_curr

array([0.01994417, 0.00739953, 0.00273066])

In [53]:
pred = net.predict(X)
pred

array([[0.48143897, 0.50816474, 0.53673459, 0.48668311, 0.46553634],
       [0.4838546 , 0.51255378, 0.52826506, 0.4860597 , 0.46110437],
       [0.4845847 , 0.51192266, 0.53278144, 0.49032614, 0.46194268],
       [0.48202594, 0.50639076, 0.53848958, 0.48758986, 0.46632257],
       [0.48642051, 0.51055976, 0.53322558, 0.49056067, 0.46009727],
       [0.48592537, 0.50560177, 0.53700116, 0.48888211, 0.46386499],
       [0.48519459, 0.51070064, 0.53378183, 0.49235269, 0.46504538],
       [0.48324058, 0.51317822, 0.53094084, 0.48851457, 0.46299257],
       [0.47723994, 0.51165853, 0.53448979, 0.48483289, 0.46764646],
       [0.47845901, 0.51377405, 0.53337973, 0.48781133, 0.46761879]])

In [56]:
seqs, labels = sample_seqs(['A', 'A', 'A', 'B'], ['True', 'True', 'True', 'False'])

ValueError: 'a' cannot be empty unless no samples are taken

In [68]:
y_hat = np.array([0.5, 0.6, 0.2, 0.3, 0.6])
y = np.array([1, 1, 1, 1, 1])
net._binary_cross_entropy_backprop(y,y_hat)

array([-0.4       , -0.33333333, -1.        , -0.66666667, -0.33333333])

In [71]:
y_hat = np.array([0.5, 0.6, 0.2, 0.3, 0.6])
y = np.array([1, 1, 1, 1, 1])
net._mean_squared_error(y,y_hat)

0.34